In [ ]:
import json
import os
import geopandas
import pandas as pd
import numpy as np
import plotly.express as px
DATE = 'date'
AREA = 'area'
px.set_mapbox_access_token(os.environ['MAPBOXKEY_CAVAX'])

In [ ]:
df_7day_orig, df_14day_orig = [
    pd.read_csv(
        f'LA_County_Covid19_CSA_{x}day_case_death_table.csv',
        parse_dates=['ep_date'],
        infer_datetime_format=True
    )
    .drop(columns='Unnamed: 0')
    .rename(columns={'ep_date': DATE, 'geo_merge': AREA})
    .sort_values([DATE, AREA])
    .reset_index(drop=True)
    for x in (7, 14)
]
df_7day_orig = df_7day_orig[df_7day_orig[DATE].notna()]
df_14day_orig = df_14day_orig[df_14day_orig[DATE].notna()]
for df in (df_7day_orig, df_14day_orig):
    for col in ('case_rate_unstable', 'death_rate_unstable'):
        df[col] = (df[col]=='^').astype(bool)
    df[AREA] = df[AREA].convert_dtypes()
    df['population'] = df['population'].astype('int')
for col in df_7day_orig:
    if '7day' in col:
        df_7day_orig[col] = (df_7day_orig[col] / 7).round(1)
for col in df_14day_orig:
    if '14day' in col:
        df_14day_orig[col] = (df_14day_orig[col] / 14).round(1)

In [ ]:
# df_csa = (
#     geopandas.read_file('lac-csa.geojson')
#     [['LABEL', 'geometry']]
#     .copy()
#     .rename(columns={'LABEL': AREA})
# )
# df_csa[AREA] = df_csa[AREA].convert_dtypes()
# df_csa = df_csa.set_index(AREA)
with open('csa.geojson') as f:
    lac_csa = json.load(f)
    

In [ ]:
df_plot = df_14day_orig[[DATE, AREA, 'case_14day_rate']].copy()
df_plot = df_plot[df_plot[AREA].isin(df_csa.index.unique())]
df_plot = df_plot[df_plot[DATE]==df_plot[DATE].max()]
# df_plot = df_plot[df_plot[AREA]!='Unincorporated - Stevenson Ranch']
# df_plot = df_plot[df_plot[AREA].apply(lambda x: 'Unincorporated' in x)]
# df_plot = df_plot[df_plot[AREA].isin(test_areas)]
# df_plot = df_plot[df_plot[AREA]=='City of Lancaster']
# df_plot = df_plot[df_plot[DATE].apply(lambda x: x.day == 1)]
# df_plot[DATE] = df_plot[DATE].apply(lambda x: x.strftime('%Y-%m-%d'))